In [ ]:
import requests
import json
import os
import time
from openai import OpenAI
from google.colab import userdata
import pandas as pd
import pprint

In [ ]:
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
news_key = userdata.get('NEWS_API_KEY')
client = OpenAI()

In [ ]:
health_query = 'Health'
health_url = (f'https://content.guardianapis.com/search?q={health_query}&api-key={news_key}&page-size=25&page=1')
health_response = requests.get(health_url)
health_data = health_response.json()

In [ ]:
pprint.pp(health_data)

In [ ]:
finance_query = 'Finance'
finance_url = (f'https://content.guardianapis.com/search?q={finance_query}&api-key={news_key}&page-size=25&page=1')
finance_response = requests.get(finance_url)
finance_data = finance_response.json()

In [ ]:
pprint.pp(finance_data)

In [ ]:
tech_query = 'Technology'
tech_url = (f'https://content.guardianapis.com/search?q={tech_query}&api-key={news_key}&page-size=25&page=1')
tech_response = requests.get(tech_url)
tech_data = tech_response.json()

In [ ]:
pprint.pp(tech_data)

In [ ]:
science_query = 'Science'
science_url = (f'https://content.guardianapis.com/search?q={science_query}&api-key={news_key}&page-size=25&page=1')
science_response = requests.get(science_url)
science_data = science_response.json()

In [ ]:
pprint.pp(science_data)

In [ ]:
dog_query = 'Dog'
dog_url = (f'https://content.guardianapis.com/search?q={dog_query}&api-key={news_key}&page-size=25&page=1')
dog_response = requests.get(dog_url)
dog_data = dog_response.json()

In [ ]:
pprint.pp(dog_data)

In [ ]:
articles =  health_data['response']['results'] + finance_data['response']['results'] + tech_data['response']['results'] + science_data['response']['results'] + dog_data['response']['results']

In [ ]:
!pip install boto3

In [ ]:
import os
import boto3
from botocore.config import Config
from botocore import UNSIGNED

In [ ]:
TEAM = "TEAM_3/"

BUCKET_NAME = "cus635-spring2025"

s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))

In [ ]:
for i, article in enumerate(articles, 1):
    # Generate a filename (using the article ID or index)
    article_id = article["id"].replace("/", "_")  # Replace slashes with underscores for safe filenames
    filename = f"{article_id}.json"

    # Convert article to JSON string
    article_json = json.dumps(article, indent=4)

     # Upload to S3
    s3.put_object(
        Bucket=BUCKET_NAME,
        Key=TEAM+filename,
        Body=article_json,
        ContentType="application/json"
    )

    print(f"Uploaded: {filename} to S3 bucket: {BUCKET_NAME}")

In [ ]:
response = s3.list_objects_v2(Bucket=BUCKET_NAME)
if "Contents" in response:
    print("Files in S3 Bucket:")
    for obj in response["Contents"]:
        print(f"- {obj['Key']}")
else:
    print("No files found in the bucket.")